In [1]:
import tensorflow as tf
from tensorflow import keras
device = tf.config.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(device, True)
print(f'TF Version:{tf.__version__}  |  GPU:{device}')

TF Version:2.2.0  |  GPU:PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
#Paths
from pathlib import Path
base_folder = Path('.')
data_folder = base_folder/'til2020'
train_imgs_folder = data_folder/'train'/'train'
train_annotations = data_folder/'train.json'
val_imgs_folder = data_folder/'val'/'val'
val_annotations = data_folder/'val.json'

train_pickle = data_folder/'train.p'/'train.p'
val_pickle = data_folder/'val.p'/'val.p'

save_model_folder = base_folder/'ckpts'
load_model_folder = base_folder/'ckpts'

In [3]:
class Config():
    cat_list = ['tops', 'trousers', 'outerwear', 'dresses', 'skirts']

    input_shape = (224,224,3)
    wt_decay = 5e-4

    dims_list = [(7,7),(14,14)]
    aspect_ratios = [(1,1), (1,2), (2,1)]

    batch_size = 16
    epoch_warmup = 300
    epoch_finetune = 300
conf = Config()

In [4]:
from tensorflow.keras import losses
# Shape of ypred: ( batch, i, j, aspect_ratios, 1+4+numclasses ). For a batch,i,j, we get #aspect_ratios vectors of length 7.
# Shape of ytrue: ( batch, i, j, aspect_ratios, 1+4+numclasses+2 ). For a batch,i,j, we get #aspect_ratios vectors of length 9 (two more for objectness and cat/loc indicators)
#TODO Play with weights?
def custom_loss(ytrue, ypred):
    obj_loss_weight = 1.0
    cat_loss_weight = 1.0
    loc_loss_weight = 1.0

    end_cat = len(conf.cat_list) + 1

    objloss_indicators = ytrue[:,:,:,:,-2:-1]
    catlocloss_indicators = ytrue[:,:,:,:,-1:]

    ytrue_obj, ypred_obj = ytrue[:,:,:,:,:1], ypred[:,:,:,:,:1]
    ytrue_obj = tf.where( objloss_indicators != 0, ytrue_obj, 0 )
    ypred_obj = tf.where( objloss_indicators != 0, ypred_obj, 0 )
    objectness_loss = losses.BinaryCrossentropy(from_logits=True)( ytrue_obj, ypred_obj )

    ytrue_cat, ypred_cat = ytrue[:,:,:,:,1:end_cat], ypred[:,:,:,:,1:end_cat]
    ytrue_cat = tf.where( catlocloss_indicators != 0, ytrue_cat, 0 )
    ypred_cat = tf.where( catlocloss_indicators != 0, ypred_cat, 0 )
    categorical_loss = losses.CategoricalCrossentropy(from_logits=True) ( ytrue_cat, ypred_cat )

    # Remember that ytrue is longer than ypred, so we will need to stop at index -2, which is where the indicators are stored
    ytrue_loc, ypred_loc = ytrue[:,:,:,:,end_cat:-2], ypred[:,:,:,:,end_cat:]
    ytrue_loc = tf.where( catlocloss_indicators != 0, ytrue_loc, 0 )
    ypred_loc = tf.where( catlocloss_indicators != 0, ypred_loc, 0 )
    localisation_loss = losses.Huber() ( ytrue_loc, ypred_loc )

    return obj_loss_weight*objectness_loss + cat_loss_weight*categorical_loss + loc_loss_weight*localisation_loss

In [5]:
#ah functional paradigm
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
#I wrote this to reduce code size (sequential layers with activation of ReLU)
def seq_with_activation(lst):
    def wrapper(x):
        nonlocal lst
        try: iter(lst)
        except TypeError: lst = [lst]
        for l in lst:
            x = l(x)
            x = layers.BatchNormalization()(x)
            x = layers.LeakyReLU(0.01)(x)
        return x
    return wrapper


def transfer_model_7x7_14x14(backbone_model, input_shape, dims_list, num_aspect_ratios, num_classes, wt_decay, model_name='transfer-objdet-model-7x7-14x14'):
    inputs = keras.Input(shape=input_shape)
    intermediate_layer_model = keras.Model(inputs=backbone_model.input,
        #outputs=backbone_model.get_layer('conv4_block6_out').output #Resnet50
        outputs=backbone_model.get_layer('block13_sepconv2_bn').output #Xceptionnet, copied example in picking last layer of res 14
        #TODO: PUT MORE THOUGHT INTO WHICH LAYER TO PICK BY INVESTIGATING ACTIVATIONS
    )

    intermediate_output = intermediate_layer_model(inputs) #14
    backbone_output = backbone_model(inputs) #7

    #TODO: not copy example, and strategize our own stuff
    upsample = seq_with_activation([
        layers.Conv2D(512, 1, padding='same', kernel_regularizer=l2(wt_decay)), #7
        layers.Conv2D(1024, 3, padding='same', kernel_regularizer=l2(wt_decay)), #7
        layers.Conv2D(512, 1, padding='same', kernel_regularizer=l2(wt_decay)), #7
        layers.Conv2D(1024, 3, padding='same', kernel_regularizer=l2(wt_decay)), #7
        layers.Conv2D(512, 1, padding='same', kernel_regularizer=l2(wt_decay)), #7
    ])(backbone_output)

    x = seq_with_activation([
        layers.Conv2D(256, 1, padding='same', kernel_regularizer=l2(wt_decay)), #7
        layers.Conv2DTranspose(512, 5, strides=(2, 2), padding='same'), #14
    ])(upsample)
    x = layers.Concatenate()([x,intermediate_output])

    tens_14x14 = seq_with_activation([
        layers.Conv2D(256, 1, padding='same', kernel_regularizer=l2(wt_decay)), #14
        layers.Conv2D(512, 3, padding='same', kernel_regularizer=l2(wt_decay)), #14
        layers.Conv2D(256, 1, padding='same', kernel_regularizer=l2(wt_decay)), #14
        layers.Conv2D(512, 3, padding='same', kernel_regularizer=l2(wt_decay)), #14
        layers.Conv2D(256, 1, padding='same', kernel_regularizer=l2(wt_decay)), #14
        layers.Conv2D(512, 3, padding='same', kernel_regularizer=l2(wt_decay)), #14
    ])(x)

    tens_7x7 = layers.Add()([
        seq_with_activation(layers.Conv2D(2048, 3, padding='same', kernel_regularizer=l2(wt_decay)))(upsample),
        backbone_output
    ])

    dim_tensor_map = {'7x7':tens_7x7,'14x14':tens_14x14}

    #Accumulate predictions for 7x7,14x14 into a dictionary for keras multi labels.
    preds_dict = {}
    for dims in dims_list:
        dimkey = '{}x{}'.format(*dims)
        tens = dim_tensor_map[dimkey]
        ar_preds = []
        for _ in range(num_aspect_ratios):
            objectness_preds = layers.Conv2D(1, 1, kernel_regularizer=l2(wt_decay))( tens )
            class_preds = layers.Conv2D(num_classes, 1, kernel_regularizer=l2(wt_decay))( tens )
            bbox_preds = layers.Conv2D(4, 1, kernel_regularizer=l2(wt_decay))( tens )
            ar_preds.append( layers.Concatenate()([objectness_preds, class_preds, bbox_preds]) )

        if num_aspect_ratios > 1: predictions = layers.Concatenate()(ar_preds)
        elif num_aspect_ratios == 1: predictions = ar_preds[0]

        predictions = layers.Reshape( (*dims, num_aspect_ratios, 5+num_classes), name=dimkey )(predictions)
        preds_dict[dimkey] = predictions

    model = keras.Model(inputs, preds_dict, name=model_name)

    model.compile( optimizer=keras.optimizers.Adam(1e-5),
                    loss=custom_loss )
    return model

In [6]:
def model_monitors(save_path):
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=save_path,
        save_weights_only=False,
        monitor='val_loss',
        mode='auto',
        save_best_only=True
    )
    earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8)
    return [model_checkpoint_callback,earlystopping,reduce_lr]

def train(model,backbone_name,warmup=True):
    if warmup: 
        save_model_path = str(save_model_folder/f'pt-{model_context}-best_val_loss.h5')
        for layer in model.get_layer(backbone_name).layers: layer.trainable = False #dont train pretrained during warm up
    else:
        save_model_path = str(save_model_folder/f'ft-{model_context}-best_val_loss.h5')
        for layer in model.get_layer(backbone_name).layers: layer.trainable = True

    model.fit(
        x=train_sequence, 
        epochs=(conf.epoch_warmup if warmup else conf.epoch_finetune), 
        validation_data=val_sequence, 
        callbacks=model_monitors(save_model_path),
        verbose=1
    )

In [7]:
model_context = 'model-7x7-14x14-3aspect-modyoloposneg-wd{}'.format(conf.wt_decay)
# load_model_path = os.path.join( load_model_folder, '{}-best_val_loss.h5'.format(model_context) )
load_model_path = None

if load_model_path is None:
    #backbone_model = keras.applications.ResNet50(input_shape=conf.input_shape,include_top=False)
    backbone_model = keras.applications.Xception(input_shape=conf.input_shape,include_top=False)
    model = transfer_model_7x7_14x14(backbone_model,
        input_shape=conf.input_shape,
        dims_list=conf.dims_list,
        num_aspect_ratios=len(conf.aspect_ratios),
        num_classes=len(conf.cat_list),
        wt_decay=conf.wt_decay,
        #model_name=model_context+'-res50'
        model_name=model_context+'-xception'
    )
else:
    model = keras.models.load_model(load_model_path, custom_objects={'custom_loss':custom_loss})

model.summary()

Model: "model-7x7-14x14-3aspect-modyoloposneg-wd0.0005-xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 7, 7, 2048)   20861480    input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 7, 7, 512)    1049088     xception[1][0]                   
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 7, 7, 512)    2048        conv2d_4[0][0]                   
____________________________________________

In [8]:
from scripts.loader import TILSequence,TILPickle
from scripts.sampling import iou,modified_yolo_posneg_sampling
from scripts.augment import aug_default,aug_identity
from scripts.encoder import encode_label

label_encoder = lambda y: encode_label(y, conf.dims_list, conf.aspect_ratios, iou, modified_yolo_posneg_sampling, conf.cat_list)
preproc_fn = lambda x: x / 255.

if True:
    train_sequence = TILPickle(train_pickle, conf.batch_size, aug_default, (224,224), label_encoder, preproc_fn)
    val_sequence = TILPickle(val_pickle, conf.batch_size, aug_identity, (224,224), label_encoder, preproc_fn)
else:
    train_sequence = TILSequence(train_imgs_folder,train_annotations,conf.batch_size,aug_default,(224,224),label_encoder,preproc_fn)
    val_sequence = TILSequence(val_imgs_folder,val_annotations,conf.batch_size,aug_identity,(224,224),label_encoder,preproc_fn)

In [9]:
print('Warming up the model...')
train(model,'xception',warmup=True)

# Fine tuning
print('Model warmed. Loading best val version of model...')
del model
load_model_path = load_model_folder/f'pt-{model_context}-best_val_loss.h5'
model = keras.models.load_model(load_model_path, custom_objects={'custom_loss':custom_loss})
model.compile(optimizer=keras.optimizers.Adam(1e-5),loss=custom_loss)
train(model,'xception',warmup=False)

# Final save
model.save(os.path.join(save_model_folder, 'ft-{}-final.h5'.format(model_context)))

Warming up the model...
Epoch 1/300
515/515 [==============================] - ETA: 0s - loss: 3.7852 - 14x14_loss: 0.2220 - 7x7_loss: 0.1532

AttributeError: 'WindowsPath' object has no attribute 'format'

In [10]:
#TODO: import hyperopts. play with hyperopts

83689472/83683744 [==============================] - 8s 0us/step
